In [83]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.simplefilter('ignore')
sns.set(rc={'figure.figsize' : (12, 6)})
sns.set_style("darkgrid", {'axes.grid' : True})

%matplotlib inline

In [84]:
tw_data0501 = pd.read_csv('twitter_data_2020-05-01_to_2020-05-09.csv', encoding = 'UTF-8')
tw_data0501.head()

,date,user_name,text
0,2020-05-09,digital_trans4m,2020 Current trends in IoT at Workplace Market...
1,2020-05-09,SoftClouds,#IoT Plays an Important Role in #Pharmaceutica...
2,2020-05-09,ajohnsocyber,Microsoft Shells Out $100K for IoT Security | ...
3,2020-05-09,KirkDBorne,With billions & billions of #IoT connected dev...
4,2020-05-09,MyTechMusings,Interesting discussion. No doubt #eSIM is the ...


In [85]:
tw_data0510 = pd.read_csv('twitter_data_2020-05-10_to_2020-05-20.csv', encoding = 'UTF-8')
tw_data0510.head()

,date,user_name,text
0,2020-05-20,IoTWorldSeries,You're invited to our next IoT World Virtual E...
1,2020-05-20,sigmaridge,Big Security in a Small Business World: 10 myt...
2,2020-05-20,ARNnet,Dicker Data expands VPN and IoT security solut...
3,2020-05-20,energyetc,"“Lighting, heating, cooling and ventilation, s..."
4,2020-05-20,sigmaridge,Education Now: Let’s Talk Distance Learning ht...


In [86]:
tw_data0521 = pd.read_csv('twitter_data_2020-05-21_to_2020-05-30.csv', encoding = 'UTF-8')
tw_data0521.head()

,date,user_name,text
0,2020-05-30,digital_trans4m,SC Awards Europe 2020 - Best IOT/IIOT Security...
1,2020-05-30,iotforall,Ignoring #Cybersecurity threats posed by insid...
2,2020-05-30,5GSec,How #5G Will Transform Economy and Society - h...
3,2020-05-30,k1rou,"Shodan founder John Matherly on IoT security, ..."
4,2020-05-30,RobTiffany,Getting Started with Azure IoT services: Secur...


In [87]:
tw_data0601 = pd.read_csv('twitter_data_2020-06-01_to_2020-06-10.csv', encoding = 'UTF-8')
tw_data0601.head()

,date,user_name,text
0,2020-06-10,digital_trans4m,All mobile operators vulnerable to denial of s...
1,2020-06-10,Intel_Jonathan,"Half of Enterprises Have No Dedicated Staff, P..."
2,2020-06-10,SuriyaSubraman,IoT Security Market How the Industry Will Witn...
3,2020-06-10,UK_CIF,Government to fund nine advanced security proj...
4,2020-06-10,paulinetaylor85,"Half of Enterprises Have No Dedicated Staff, P..."


In [88]:
#quora데이터
q_data = pd.read_csv('quora_data.csv', encoding = 'latin')
q_data.rename(columns = {'comments' : 'text'}, inplace = True)
q_data.head()

,text
0,Research Study forecasts revenue from the glob...
1,Download Sample PDF to get More Information ab...
2,Government initiatives in smart cities projects
3,Here is a list of some root cause for vulnerab...
4,"According to New IDC Spending Guide, the world..."


In [89]:
#해외 커뮤니티에 올려서 수집한 설문조사 데이터
survey = pd.read_csv("survey result.csv", encoding = 'latin')
survey.columns = ['date','do you know iotsec?','are you expert?', 'sentiment','text']
survey.head()

,date,do you know iotsec?,are you expert?,sentiment,text
0,2020/06/05 3:07:17 ?¤í GMT+9,Yes,Yes,Negative,The human factor
1,2020/06/05 11:54:13 ?¤í GMT+9,Yes,Yes,Negative,See my quora response
2,2020/06/06 12:08:03 ?¤ì  GMT+9,Yes,No,Neutral,It seems important but I don't know what I hav...
3,2020/06/06 1:09:09 ?¤ì  GMT+9,Yes,No,Negative,"I feel that a lot of IoT devices are insecure,..."
4,2020/06/06 2:06:00 ?¤ì  GMT+9,Yes,Yes,Neutral,Can't over simplify the epic generalisation go...


In [90]:
s_data = pd.DataFrame(columns = ['text'])
s_data['text'] = survey['text']
s_data.head()

,text
0,The human factor
1,See my quora response
2,It seems important but I don't know what I hav...
3,"I feel that a lot of IoT devices are insecure,..."
4,Can't over simplify the epic generalisation go...


In [91]:
tw_data = pd.concat([tw_data0501, tw_data0510, tw_data0521, tw_data0601])
tw_data.drop(['date', 'user_name'], axis='columns', inplace=True)
tw_data.head()

,text
0,2020 Current trends in IoT at Workplace Market...
1,#IoT Plays an Important Role in #Pharmaceutica...
2,Microsoft Shells Out $100K for IoT Security | ...
3,With billions & billions of #IoT connected dev...
4,Interesting discussion. No doubt #eSIM is the ...


In [92]:
total_data = pd.concat([tw_data, q_data, s_data])
total_data.head()

,text
0,2020 Current trends in IoT at Workplace Market...
1,#IoT Plays an Important Role in #Pharmaceutica...
2,Microsoft Shells Out $100K for IoT Security | ...
3,With billions & billions of #IoT connected dev...
4,Interesting discussion. No doubt #eSIM is the ...


In [93]:
t_data = pd.DataFrame(columns = ['review','sentiment'])
t_data['review'] = total_data['text']
t_data['sentiment'] = 0
t_data.head()

,review,sentiment
0,2020 Current trends in IoT at Workplace Market...,0
1,#IoT Plays an Important Role in #Pharmaceutica...,0
2,Microsoft Shells Out $100K for IoT Security | ...,0
3,With billions & billions of #IoT connected dev...,0
4,Interesting discussion. No doubt #eSIM is the ...,0


In [94]:
import string
import re


def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

t_data['review'] = t_data['review'].str.replace('<br />',' ')
t_data['review'] = t_data['review'].apply(lambda x:clean_text(x))
t_data.head()

,review,sentiment
0,current trends in iot at workplace market is ...,0
1,iot plays an important role in pharmaceutical ...,0
2,microsoft shells out for iot security threat...,0
3,with billions billions of iot connected devic...,0
4,interesting discussion no doubt esim is the fu...,0


In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def ngram_vectorize(texts, labels):
    kwargs = {'ngram_range' : (1, 2),'dtype' : 'int32','strip_accents' : 'unicode',
              'decode_error' : 'replace','analyzer' : 'word','min_df' : 2}

    #tf-idf로 vectorize
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    transformed_texts = tfidf_vectorizer.fit_transform(texts)
    
    # selector를 이용하여, vector화된 값들을 최적화  
    selector = SelectKBest(f_classif, k=min(20000, transformed_texts.shape[1]))
    selector.fit(transformed_texts, labels)
    transformed_texts = selector.transform(transformed_texts).astype('float32')

    return transformed_texts

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer

#벡터화
tfidf = TfidfVectorizer()
tr_texts = tfidf.fit_transform(t_data['review'])
vect_data = ngram_vectorize(t_data['review'], t_data['sentiment'])

X = vect_data.toarray()

#준비된 모델을 통한 결과 예측
model = keras.models.load_model('IMDB_model.h5')
result = model.predict(X)

In [97]:
result.shape

(12546, 1)

In [98]:
t_data.head(20)

,review,sentiment
0,current trends in iot at workplace market is ...,0
1,iot plays an important role in pharmaceutical ...,0
2,microsoft shells out for iot security threat...,0
3,with billions billions of iot connected devic...,0
4,interesting discussion no doubt esim is the fu...,0
5,detecting and mitigating iot breaches require ...,0
6,i blocked ads that is of my internet traffic...,0
7,kubernetes cisco container platform petascale...,0
8,sources microsoft to buy israelbased cyberx wh...,0
9,apimote for zigbee sniffing and transmission ...,0


In [99]:
output = pd.DataFrame(columns = ['review', 'sentiment'])
output['review'] = total_data['text']
output['sentiment'] = result
output.head(20)

,review,sentiment
0,2020 Current trends in IoT at Workplace Market...,0.324111
1,#IoT Plays an Important Role in #Pharmaceutica...,0.214899
2,Microsoft Shells Out $100K for IoT Security | ...,0.038622
3,With billions & billions of #IoT connected dev...,0.009276
4,Interesting discussion. No doubt #eSIM is the ...,0.951570
5,Detecting and Mitigating IoT Breaches Require ...,0.440807
6,I blocked 1568 ads. That is 0.73% of my intern...,0.985567
7,Kubernetes + Cisco Container Platform: Petasca...,0.935234
8,"Sources: Microsoft to buy Israel-based CyberX,...",0.963232
9,APIMOTE (FOR ZIGBEE SNIFFING AND TRANSMISSION)...,0.024169


In [100]:
output.to_csv('sentiment_result.csv', mode='w')

In [101]:
print(len(total_data))
print(len(tw_data0501))
print(len(tw_data0510))
print(len(tw_data0521))
print(len(tw_data0601))
print(len(q_data))
print(len(s_data))

12546
2577
3234
2972
3364
360
39


In [102]:
output['sentiment'].describe()

count    12546.000000
mean         0.406363
std          0.388691
min          0.000044
25%          0.031036
50%          0.260707
75%          0.835618
max          0.999875
Name: sentiment, dtype: float64

In [103]:
output['sentiment'][:2577].describe()

count    2577.000000
mean        0.405233
std         0.392940
min         0.000111
25%         0.031606
50%         0.251772
75%         0.846413
max         0.999858
Name: sentiment, dtype: float64

In [104]:
output['sentiment'][2577:5811].describe()

count    3234.000000
mean        0.423869
std         0.394743
min         0.000044
25%         0.032926
50%         0.288355
75%         0.881827
max         0.999835
Name: sentiment, dtype: float64

In [81]:
output['sentiment'][-399:-39].describe()

count    360.000000
mean       0.452885
std        0.397004
min        0.000085
25%        0.037638
50%        0.405298
75%        0.919398
max        0.998462
Name: sentiment, dtype: float64

In [82]:
output['sentiment'][-39:].describe()

count    39.000000
mean      0.331219
std       0.291376
min       0.003809
25%       0.080527
50%       0.251068
75%       0.414764
max       0.967888
Name: sentiment, dtype: float64